# 라이브러리 호출

In [1]:
import numpy as np # Numpy
import pandas as pd # Pandas
import matplotlib as mpl #Matplotlib 세팅용
import matplotlib.pyplot as plt # 시각화 도구
import seaborn as sns # 시각화 도구
from sklearn.model_selection import train_test_split # 데이터셋 분리
from sklearn.model_selection import KFold # KFold 교차검증
from sklearn.cluster import KMeans # 클러스터링
from sklearn.metrics import silhouette_score # 실루엣 점수
import xgboost as xgb # XGBoost
from sklearn.model_selection import GridSearchCV # 그리드 서치
from sklearn.metrics import accuracy_score, precision_score # 평가 지표
from sklearn.metrics import recall_score, confusion_matrix, roc_auc_score, f1_score # 평가 지표
from imblearn.combine import SMOTEENN, SMOTETomek # 복합샘플링
from hyperopt import hp, fmin, tpe, Trials # HyperOPT
from nltk.corpus import names # nltk
import nltk
nltk.download("names")
from nltk import NaiveBayesClassifier
from scipy import stats
from collections import Counter
import random

import warnings # 경고문 제거용


%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

# 한글 폰트 설정
mpl.rc('font', family='D2Coding')
# 유니코드에서 음수 부호 설정
mpl.rc('axes', unicode_minus = False)

warnings.filterwarnings('ignore')
sns.set(font="D2Coding", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc('figure', figsize=(10,8))

[nltk_data] Downloading package names to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!


# 데이터로딩

In [2]:
data = pd.read_excel('train_test_na_filled.xlsx', sheet_name='Train')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   bool   
 3   Cabin1        8590 non-null   object 
 4   Cabin2        8590 non-null   float64
 5   Combi         8590 non-null   object 
 6   Cabin3        8590 non-null   object 
 7   Cabin         8590 non-null   object 
 8   Destination   8693 non-null   object 
 9   Age           8693 non-null   int64  
 10  VIP           8693 non-null   bool   
 11  RoomService   8693 non-null   int64  
 12  FoodCourt     8693 non-null   int64  
 13  ShoppingMall  8693 non-null   int64  
 14  Spa           8693 non-null   int64  
 15  VRDeck        8693 non-null   int64  
 16  Name          8493 non-null   object 
 17  Transported   8693 non-null   bool   
dtypes: bool(3), float64(1), int6

In [3]:
test = pd.read_excel('train_test_na_filled.xlsx', sheet_name='Test')
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin1,Cabin2,Combi,Cabin3,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0013_01,Earth,True,G,3.0,G3,S,G/3/S,TRAPPIST-1e,27,False,0,0,0,0,0,Nelly Carsoning,NaN
1,0018_01,Earth,False,F,4.0,F4,S,F/4/S,TRAPPIST-1e,19,False,0,9,0,2823,0,Lerome Peckers,NaN
2,0019_01,Europa,True,C,0.0,C0,S,C/0/S,55 Cancri e,31,False,0,0,0,0,0,Sabih Unhearfus,NaN
3,0021_01,Europa,False,C,1.0,C1,S,C/1/S,TRAPPIST-1e,38,False,0,6652,0,181,585,Meratz Caltilter,NaN
4,0023_01,Earth,False,F,5.0,F5,S,F/5/S,TRAPPIST-1e,20,False,10,0,635,0,0,Brence Harperez,NaN


# 탐색

## ANOVA 분석

In [4]:
numeric_data = [column for column in data.select_dtypes(["int", "float"])]

for column in numeric_data:
  df_anova = data[[column,'Transported']]
  grouped_anova = df_anova.groupby(['Transported'])
  f_value, p_value = stats.f_oneway(grouped_anova.get_group(True)[column],
                                    grouped_anova.get_group(False)[column])
  result = ""
  if p_value < 0.05:
    result = "{} is important feature for prediction".format(column)
  else:
    result = "{} is not an important feature for prediction".format(column)
  print(result)

Cabin2 is not an important feature for prediction
Age is important feature for prediction
RoomService is important feature for prediction
FoodCourt is important feature for prediction
ShoppingMall is not an important feature for prediction
Spa is important feature for prediction
VRDeck is important feature for prediction


In [5]:
def outlier_detection_train(df, n, columns):
    rows = []
    will_drop_train = []
    for col in columns:
        Q1 = np.nanpercentile(df[col], 25)
        Q3 = np.nanpercentile(df[col], 75)
        IQR = Q3 - Q1
        outlier_point = 1.5 * IQR
        rows.extend(df[(df[col] < Q1 - outlier_point)|(df[col] > Q3 + outlier_point)].index)
    for r, c in Counter(rows).items():
        if c >= n: will_drop_train.append(r)
    return will_drop_train

In [6]:
data.drop('Cabin2', inplace=True, axis=1)

In [7]:
test.drop('Cabin2', inplace=True, axis=1)

In [8]:
data.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin1', 'Combi', 'Cabin3',
       'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt',
       'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported'],
      dtype='object')

# 또처리~

## 이상치 확인 및 제거

In [9]:
def outlier_detection_train(df, n, columns):
    rows = []
    will_drop_train = []
    for col in columns:
        Q1 = np.nanpercentile(data[col], 25)
        Q3 = np.nanpercentile(data[col], 75)
        IQR = Q3 - Q1
        outlier_point = 1.5 * IQR
        rows.extend(df[(df[col] < Q1 - outlier_point)|(df[col] > Q3 + outlier_point)].index)
    for r, c in Counter(rows).items():
        if c >= n: will_drop_train.append(r)
    return will_drop_train

In [10]:
will_drop_train = outlier_detection_train(data, 5 ,data.select_dtypes(["float", "int"]).columns)
will_drop_train

[338,
 1390,
 6469,
 7038,
 1936,
 3317,
 3980,
 4762,
 6509,
 7007,
 7065,
 7294,
 7689,
 7957,
 8064]

In [11]:
data.drop(will_drop_train, inplace = True, axis = 0)

## 새로운 feature 생성

### 총 사용금액, 그리고 사용한 금액에 따라 poor, middle, rich로 분류

In [12]:
data["Total"] = data["RoomService"] + data["FoodCourt"] + data["ShoppingMall"] + data["Spa"] +\
data["VRDeck"] 
data["RichPoor"] = data["Total"].apply(lambda x: "poor" if x < 5000\
                                       else ("middle" if x>=5000 and x<20000 else "rich"))

test["Total"] = test["RoomService"] + test["FoodCourt"] + test["ShoppingMall"] + test["Spa"] +\
test["VRDeck"] 
test["RichPoor"] = test["Total"].apply(lambda x: "poor" if x < 5000\
                                       else ("middle" if x>=5000 and x<20000 else "rich"))

### 그룹여행객 여부

In [13]:
data["GroupId"] = data["PassengerId"].apply(lambda x: x.split("_")[0])
test["GroupId"] = test["PassengerId"].apply(lambda x: x.split("_")[0])
data["GroupNo"] = data["PassengerId"].apply(lambda x: x.split("_")[1])
test["GroupNo"] = test["PassengerId"].apply(lambda x: x.split("_")[1])

train_g = data[data["GroupId"].duplicated()]["GroupId"]
test_g = test[test["GroupId"].duplicated()]["GroupId"]
data["Group"] = data["GroupId"].apply(lambda x: x in train_g.values)
test["Group"] = test["GroupId"].apply(lambda x: x in test_g.values)

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8678 entries, 0 to 8692
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PassengerId   8678 non-null   object
 1   HomePlanet    8678 non-null   object
 2   CryoSleep     8678 non-null   bool  
 3   Cabin1        8575 non-null   object
 4   Combi         8575 non-null   object
 5   Cabin3        8575 non-null   object
 6   Cabin         8575 non-null   object
 7   Destination   8678 non-null   object
 8   Age           8678 non-null   int64 
 9   VIP           8678 non-null   bool  
 10  RoomService   8678 non-null   int64 
 11  FoodCourt     8678 non-null   int64 
 12  ShoppingMall  8678 non-null   int64 
 13  Spa           8678 non-null   int64 
 14  VRDeck        8678 non-null   int64 
 15  Name          8478 non-null   object
 16  Transported   8678 non-null   bool  
 17  Total         8678 non-null   int64 
 18  RichPoor      8678 non-null   object
 19  GroupI

### 나이브 베어스를 활용한 이름을 통한 성별 분류

In [15]:
# Train_Data
names_train_data = []
for n in data["Name"]:
    n = str(n)
    a = n.split()
    names_train_data.append(a[0])

#### 훈련셋

In [16]:
# 이름과 성 분리
names_train_data = []
for i in data["Name"]:
    i = str(i)
    a = i.split()
    names_train_data.append(a[0])

In [17]:
# NLTK의 names 파일을 활용하여 이름을 여성과 남성으로 분리
labeled_names = [(name, "female") for name in names.words("female.txt")] + \
[(name, "male") for name in names.words("male.txt")]
random.shuffle(labeled_names)

In [18]:
# 이름의 마지막 단어 가져오는 함수
def gender_features(word):
    return {'last_letter': word[-1]}

In [19]:
names_train_data

['Maham',
 'Juanna',
 'Altark',
 'Solam',
 'Willy',
 'Sandie',
 'Billex',
 'Candra',
 'Andona',
 'Erraiam',
 'Altardr',
 'Wezena',
 'Berers',
 'Reney',
 'Elle',
 'Justie',
 'Flats',
 'Carry',
 'Alus',
 'Lyde',
 'Philda',
 'Almary',
 'Glendy',
 'Mollen',
 'Breney',
 'Mael',
 'Terta',
 'nan',
 'Penton',
 'Karard',
 'Anyoni',
 'Ceros',
 'Ginia',
 'Coobix',
 'Cinets',
 'Dontch',
 'Ziba',
 'Luse',
 'Marina',
 'Loise',
 'Jorgie',
 'Margia',
 'Ankalik',
 'Jodye',
 'Kayne',
 'Cassa',
 'Zelowl',
 'Mass',
 'Sony',
 'Vivia',
 'Elaney',
 'Elson',
 'Okulas',
 'Instab',
 'Zinoces',
 'Warry',
 'Shanya',
 'Sterry',
 'nan',
 'Colatz',
 'Diandy',
 'Ninaha',
 'Celine',
 'Velyne',
 'Cinst',
 'nan',
 'Meremy',
 'Nelly',
 'Thell',
 'Gorn',
 'Aldibah',
 'Conk',
 'Pon',
 'Spuri',
 'Dellie',
 'Totse',
 'Eaturs',
 'nan',
 'Coren',
 'Furudah',
 'Jodye',
 'Stmeal',
 'Heremy',
 'Deanne',
 'Tinez',
 'Gracy',
 'Stald',
 'Tiney',
 'Alchium',
 'Doria',
 'Leence',
 'Aliey',
 'Thewis',
 'Book',
 'Ritany',
 'Arlen',
 'Sh

In [20]:
# 나이브 베어스 모델 학습
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
classifier = NaiveBayesClassifier.train(featuresets)

In [21]:
# 성별 feature 생성
names_gender = []
for i in names_train_data:
    names_gender.append(classifier.classify(gender_features(i)))

# create new column called 'gender'    
data["Gender"] = names_gender

In [22]:
data.Gender[data.Name.isna()] = 'female'

In [23]:
data.Gender[data.Name.isna()].unique()

array(['female'], dtype=object)

#### 테스트셋

In [24]:
# 이름과 성 분리
names_test_data = []
for i in test["Name"]:
    i = str(i)
    a = i.split()
    names_test_data.append(a[0])

In [25]:
# NLTK의 names 파일을 활용하여 이름을 여성과 남성으로 분리
labeled_names = [(name, "female") for name in names.words("female.txt")] + \
[(name, "male") for name in names.words("male.txt")]
random.shuffle(labeled_names)

In [26]:
# 나이브 베어스 모델 학습
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
classifier = NaiveBayesClassifier.train(featuresets)

In [27]:
# 이름의 마지막 단어 가져오는 함수
def gender_features(word):
    return {'last_letter': word[-1]}

In [28]:
# 성별 feature 생성
names_gender = []
for i in names_test_data:
    names_gender.append(classifier.classify(gender_features(i)))

In [29]:
test["Gender"] = pd.Series(names_gender)

In [30]:
test.Gender[test.Name.isna()] = 'female'

In [31]:
data.Gender[data.Name.isna()].unique()

array(['female'], dtype=object)

## Cabin 결측값들 제거

In [32]:
data.dropna(axis=0, inplace=True)

## 필요없는 features 제거

In [33]:
target = data['Transported']
data = data.drop(["PassengerId", "Name", "Cabin", "Combi", "Total", "GroupId",\
                  "GroupNo", "Transported"], axis = 1)
test = test.drop(["PassengerId", "Name", "Cabin", "Combi", "Total", "GroupId",\
                  "GroupNo", "Transported"], axis = 1)


## 째려보기

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8375 entries, 0 to 8692
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   HomePlanet    8375 non-null   object
 1   CryoSleep     8375 non-null   bool  
 2   Cabin1        8375 non-null   object
 3   Cabin3        8375 non-null   object
 4   Destination   8375 non-null   object
 5   Age           8375 non-null   int64 
 6   VIP           8375 non-null   bool  
 7   RoomService   8375 non-null   int64 
 8   FoodCourt     8375 non-null   int64 
 9   ShoppingMall  8375 non-null   int64 
 10  Spa           8375 non-null   int64 
 11  VRDeck        8375 non-null   int64 
 12  RichPoor      8375 non-null   object
 13  Group         8375 non-null   bool  
 14  Gender        8375 non-null   object
dtypes: bool(3), int64(6), object(6)
memory usage: 875.1+ KB


In [35]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   HomePlanet    4277 non-null   object
 1   CryoSleep     4277 non-null   bool  
 2   Cabin1        4214 non-null   object
 3   Cabin3        4214 non-null   object
 4   Destination   4277 non-null   object
 5   Age           4277 non-null   int64 
 6   VIP           4277 non-null   bool  
 7   RoomService   4277 non-null   int64 
 8   FoodCourt     4277 non-null   int64 
 9   ShoppingMall  4277 non-null   int64 
 10  Spa           4277 non-null   int64 
 11  VRDeck        4277 non-null   int64 
 12  RichPoor      4277 non-null   object
 13  Group         4277 non-null   bool  
 14  Gender        4277 non-null   object
dtypes: bool(3), int64(6), object(6)
memory usage: 413.6+ KB


## 원핫인코딩

### boolean 타입 피처들 object로 캐스팅

In [36]:
bool_data = [column for column in data.select_dtypes(["bool"])]

bool_data

['CryoSleep', 'VIP', 'Group']

In [37]:
data["VIP"] = data["VIP"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")
data["CryoSleep"] = data["CryoSleep"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")
data["Group"] = data["Group"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8375 entries, 0 to 8692
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   HomePlanet    8375 non-null   object
 1   CryoSleep     8375 non-null   object
 2   Cabin1        8375 non-null   object
 3   Cabin3        8375 non-null   object
 4   Destination   8375 non-null   object
 5   Age           8375 non-null   int64 
 6   VIP           8375 non-null   object
 7   RoomService   8375 non-null   int64 
 8   FoodCourt     8375 non-null   int64 
 9   ShoppingMall  8375 non-null   int64 
 10  Spa           8375 non-null   int64 
 11  VRDeck        8375 non-null   int64 
 12  RichPoor      8375 non-null   object
 13  Group         8375 non-null   object
 14  Gender        8375 non-null   object
dtypes: int64(6), object(9)
memory usage: 1.0+ MB


### 더미화

In [39]:
# drop_first 첫번째 범주는 제거하고 더미화
# 다른 범주가 전부 0이면 자동적으로 첫번째 범주가 1인걸 알 수 있기에 사용
df = pd.get_dummies(data, drop_first = False)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8375 entries, 0 to 8692
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   Age                        8375 non-null   int64
 1   RoomService                8375 non-null   int64
 2   FoodCourt                  8375 non-null   int64
 3   ShoppingMall               8375 non-null   int64
 4   Spa                        8375 non-null   int64
 5   VRDeck                     8375 non-null   int64
 6   HomePlanet_Earth           8375 non-null   uint8
 7   HomePlanet_Europa          8375 non-null   uint8
 8   HomePlanet_Mars            8375 non-null   uint8
 9   CryoSleep_No               8375 non-null   uint8
 10  CryoSleep_Yes              8375 non-null   uint8
 11  Cabin1_A                   8375 non-null   uint8
 12  Cabin1_B                   8375 non-null   uint8
 13  Cabin1_C                   8375 non-null   uint8
 14  Cabin1_D                

# 데이터셋 분리

In [41]:
X_train, X_test, y_train, y_test = train_test_split(df, target, shuffle = True, random_state = 109)
X_train, X_val, y_train, y_val = train_test_split(df, target, shuffle = True,\
                                                  test_size=0.15, random_state = 109)

# XGBoost

## HyperOPT

In [42]:
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 15, 1),
                   'min_child_weight': hp.quniform('min_child_weight', 1, 20, 1),
                   'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 0.95),
                   'learning_rate': hp.uniform('learning_rate', 0.01, 0.4),
                   'gamma': hp.uniform('gamma', 0, 4)}

In [43]:
# fmin()에서 호출 시 search_space 값으로 XGBClassifier 교차 검증 학습 후 -1 * roc_auc 평균 값을 반환
def bin_objective_func(search_space):
    xgb_clf = xgb.XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            colsample_bytree=search_space['colsample_bytree'],
                            learning_rate=search_space['learning_rate'],
                            gamma=search_space['gamma'])
    
    # 3개 k-fold 방식으로 평가된 roc_auc 지표를 담는 list
    roc_auc_list = []
    
    # 3개 k-fold 방식 적용
    kf = KFold(n_splits=3)
    
    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train):
        # kf.split(X_train)으로 추출된 학습과 검증 index 값으로 학습과 검증 데이터 세트 분리
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]
        
        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행
        xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric="auc",
                   eval_set=[(X_tr, y_tr), (X_val, y_val)])
        
        # 1로 예측한 확률값 추출 후 roc auc 계산하고 평균 roc auc 계산을 위해 list에 결과값 담음.
        score = roc_auc_score(y_val, xgb_clf.predict_proba(X_val)[:,1])
        roc_auc_list.append(score)
        
    # 3개 k-fold로 계산된 roc_auc 값의 평균값을 반환하되,
    # HyperOPT는 목적함수의 최솟값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환
    return -1*np.mean(roc_auc_list)

In [44]:
trials = Trials()

# fmin() 함수를 호출. max_evals 지정된 횟수만큼 반복 후 목적함수의 최솟값을 가지는 최적 입력값 추출
best = fmin(fn=bin_objective_func,
           space=xgb_search_space,
           algo=tpe.suggest,
           max_evals=50, # 최대 반복 횟수를 지정합니다
           trials=trials, rstate=np.random.default_rng(seed=109))

print('best:', best)

[0]	validation_0-auc:0.90292	validation_1-auc:0.86834                                                                  
[1]	validation_0-auc:0.91716	validation_1-auc:0.87756                                                                  
[2]	validation_0-auc:0.92157	validation_1-auc:0.88151                                                                  
[3]	validation_0-auc:0.92476	validation_1-auc:0.88554                                                                  
[4]	validation_0-auc:0.92779	validation_1-auc:0.88456                                                                  
[5]	validation_0-auc:0.92843	validation_1-auc:0.88558                                                                  
[6]	validation_0-auc:0.93110	validation_1-auc:0.88773                                                                  
[7]	validation_0-auc:0.93433	validation_1-auc:0.88827                                                                  
[8]	validation_0-auc:0.93593	validation_

[37]	validation_0-auc:0.95472	validation_1-auc:0.88859                                                                 
[38]	validation_0-auc:0.95472	validation_1-auc:0.88859                                                                 
[39]	validation_0-auc:0.95472	validation_1-auc:0.88859                                                                 
[40]	validation_0-auc:0.95472	validation_1-auc:0.88859                                                                 
[41]	validation_0-auc:0.95574	validation_1-auc:0.88820                                                                 
[42]	validation_0-auc:0.95595	validation_1-auc:0.88792                                                                 
[43]	validation_0-auc:0.95595	validation_1-auc:0.88792                                                                 
[44]	validation_0-auc:0.95595	validation_1-auc:0.88792                                                                 
[45]	validation_0-auc:0.95595	validation

[12]	validation_0-auc:0.90210	validation_1-auc:0.88038                                                                 
[13]	validation_0-auc:0.90280	validation_1-auc:0.88087                                                                 
[14]	validation_0-auc:0.90420	validation_1-auc:0.88089                                                                 
[15]	validation_0-auc:0.90498	validation_1-auc:0.88092                                                                 
[16]	validation_0-auc:0.90632	validation_1-auc:0.88169                                                                 
[17]	validation_0-auc:0.90705	validation_1-auc:0.88205                                                                 
[18]	validation_0-auc:0.90819	validation_1-auc:0.88258                                                                 
[19]	validation_0-auc:0.90904	validation_1-auc:0.88304                                                                 
[20]	validation_0-auc:0.90952	validation

[69]	validation_0-auc:0.92391	validation_1-auc:0.89022                                                                 
[70]	validation_0-auc:0.92468	validation_1-auc:0.89042                                                                 
[71]	validation_0-auc:0.92502	validation_1-auc:0.89037                                                                 
[72]	validation_0-auc:0.92567	validation_1-auc:0.89059                                                                 
[73]	validation_0-auc:0.92599	validation_1-auc:0.89052                                                                 
[74]	validation_0-auc:0.92620	validation_1-auc:0.89057                                                                 
[75]	validation_0-auc:0.92620	validation_1-auc:0.89057                                                                 
[76]	validation_0-auc:0.92669	validation_1-auc:0.89064                                                                 
[77]	validation_0-auc:0.92669	validation

[16]	validation_0-auc:0.96278	validation_1-auc:0.88455                                                                 
[17]	validation_0-auc:0.96489	validation_1-auc:0.88285                                                                 
[18]	validation_0-auc:0.96637	validation_1-auc:0.88329                                                                 
[19]	validation_0-auc:0.96670	validation_1-auc:0.88308                                                                 
[20]	validation_0-auc:0.96761	validation_1-auc:0.88254                                                                 
[21]	validation_0-auc:0.96825	validation_1-auc:0.88282                                                                 
[22]	validation_0-auc:0.96858	validation_1-auc:0.88290                                                                 
[23]	validation_0-auc:0.96993	validation_1-auc:0.88343                                                                 
[24]	validation_0-auc:0.97006	validation

...

[1]	validation_0-auc:0.89057	validation_1-auc:0.87131                                                                  
[2]	validation_0-auc:0.89795	validation_1-auc:0.87586                                                                  
[3]	validation_0-auc:0.90285	validation_1-auc:0.88147                                                                  
[4]	validation_0-auc:0.90679	validation_1-auc:0.88242                                                                  
[5]	validation_0-auc:0.90842	validation_1-auc:0.88344                                                                  
[6]	validation_0-auc:0.91014	validation_1-auc:0.88477                                                                  
[7]	validation_0-auc:0.91235	validation_1-auc:0.88584                                                                  
[8]	validation_0-auc:0.91455	validation_1-auc:0.88716                                                                  
[9]	validation_0-auc:0.91635	validation_

[6]	validation_0-auc:0.89654	validation_1-auc:0.87693                                                                  
[7]	validation_0-auc:0.89691	validation_1-auc:0.87683                                                                  
[8]	validation_0-auc:0.89852	validation_1-auc:0.87756                                                                  
[9]	validation_0-auc:0.89890	validation_1-auc:0.87779                                                                  
[10]	validation_0-auc:0.90073	validation_1-auc:0.87888                                                                 
[11]	validation_0-auc:0.90193	validation_1-auc:0.87962                                                                 
[12]	validation_0-auc:0.90337	validation_1-auc:0.88035                                                                 
[13]	validation_0-auc:0.90414	validation_1-auc:0.88009                                                                 
[14]	validation_0-auc:0.90484	validation

[10]	validation_0-auc:0.91351	validation_1-auc:0.87596                                                                 
[11]	validation_0-auc:0.91484	validation_1-auc:0.87706                                                                 
[12]	validation_0-auc:0.91562	validation_1-auc:0.87813                                                                 
[13]	validation_0-auc:0.91537	validation_1-auc:0.87618                                                                 
[14]	validation_0-auc:0.91600	validation_1-auc:0.87648                                                                 
[15]	validation_0-auc:0.91683	validation_1-auc:0.87777                                                                 
[16]	validation_0-auc:0.91830	validation_1-auc:0.87912                                                                 
[17]	validation_0-auc:0.91893	validation_1-auc:0.87982                                                                 
[18]	validation_0-auc:0.91913	validation

[46]	validation_0-auc:0.92059	validation_1-auc:0.88380                                                                 
[47]	validation_0-auc:0.92073	validation_1-auc:0.88353                                                                 
[48]	validation_0-auc:0.92098	validation_1-auc:0.88401                                                                 
[49]	validation_0-auc:0.92136	validation_1-auc:0.88418                                                                 
[50]	validation_0-auc:0.92145	validation_1-auc:0.88433                                                                 
[51]	validation_0-auc:0.92138	validation_1-auc:0.88408                                                                 
[52]	validation_0-auc:0.92137	validation_1-auc:0.88414                                                                 
[53]	validation_0-auc:0.92151	validation_1-auc:0.88425                                                                 
[54]	validation_0-auc:0.92175	validation

[82]	validation_0-auc:0.92999	validation_1-auc:0.88659                                                                 
[83]	validation_0-auc:0.93010	validation_1-auc:0.88665                                                                 
[84]	validation_0-auc:0.93023	validation_1-auc:0.88675                                                                 
[85]	validation_0-auc:0.93043	validation_1-auc:0.88692                                                                 
[86]	validation_0-auc:0.93067	validation_1-auc:0.88692                                                                 
[87]	validation_0-auc:0.93081	validation_1-auc:0.88695                                                                 
[88]	validation_0-auc:0.93100	validation_1-auc:0.88702                                                                 
[89]	validation_0-auc:0.93115	validation_1-auc:0.88712                                                                 
[90]	validation_0-auc:0.93125	validation

## 모델 학습

In [52]:
xgbo = xgb.XGBClassifier(colsample_bytree=0.7379177001235193, gamma=3.0045640131168003,
                         learning_rate=0.23444409533390848,
                         max_depth=6, min_child_weight=4, random_state=109)
xgbo.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.7379177001235193, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=3.0045640131168003, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.23444409533390848, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=6, max_leaves=0, min_child_weight=4, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=109, ...)

## feature 중요도

In [53]:
fi = pd.DataFrame(xgbo.feature_importances_, index=df.columns)
fi.to_csv('fi_1')
fi

,0
Age,0.008890
RoomService,0.018778
FoodCourt,0.015711
ShoppingMall,0.013643
Spa,0.020228
VRDeck,0.019840
HomePlanet_Earth,0.058300
HomePlanet_Europa,0.045572
HomePlanet_Mars,0.012858
CryoSleep_No,0.531224


## 평가지표

In [54]:
# 평가용 함수
def  get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
#     roc_auc = roc_auc_score(y_test, pred_proba)
    
    print('오차 행렬')
    print(confusion)
 
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, \
    재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

In [55]:
train_pred = xgbo.predict(X_train)
train_proba = xgbo.predict_proba(X_train)

test_pred = xgbo.predict(X_test)
test_proba = xgbo.predict_proba(X_test)

val_pred = xgbo.predict(X_val)
val_proba = xgbo.predict_proba(X_val)

In [56]:
get_clf_eval(y_train, train_pred, train_proba)

오차 행렬
[[2980  548]
 [ 525 3065]]
정확도: 0.8493, 정밀도: 0.8483,     재현율: 0.8538, F1: 0.8510


In [57]:
get_clf_eval(y_test, test_pred, test_proba)

오차 행렬
[[843 192]
 [181 878]]
정확도: 0.8219, 정밀도: 0.8206,     재현율: 0.8291, F1: 0.8248


In [58]:
get_clf_eval(y_val, val_pred, val_proba)

오차 행렬
[[495 123]
 [126 513]]
정확도: 0.8019, 정밀도: 0.8066,     재현율: 0.8028, F1: 0.8047
